## Calculating similarity score using Levenshtein distance.

In [113]:
!pip install docx2txt

In [114]:
!pip install strsimpy

In [115]:
# importing all necessary libraries
import pandas as pd
import numpy as np
import docx2txt
import json, collections
from pandas.io.json import json_normalize
import os

In [116]:
!pip install docx2csv

In [117]:
from docx2csv import extract_tables, extract
tables = extract_tables('/content/lists.docx')

In [118]:
import pandas as pd
from docx import Document
document = Document("/content/lists.docx")

In [119]:
def read_docx_table(document,table_num):
  table = document.tables[table_num-1]
  data = [[cell.text for cell in row.cells] for row in table.rows]
  df = pd.DataFrame(data)
  return df

In [120]:
table_num=1
df = read_docx_table(document,table_num)
df.head()

,0,1
0,P1,Analyze a complex computing problem and to app...
1,P2,"Design, implement, and evaluate a computing-ba..."
2,P3,Communicate effectively in a variety of profes...
3,P4,Recognize professional responsibilities and ma...
4,P5,Function effectively as a member or leader of ...


In [121]:
q1 = df.copy()

In [122]:
table_num=2
df1 = read_docx_table(document,table_num)
df1.head()

,0,1
0,C1,Explain how AI impacts society
1,C2,Apply AI methods to perform practical tasks
2,C3,Synthesize a simple AI system
3,C4,Evaluate different AI approaches to solve a pr...
4,C5,Explain the basic principles of the data minin...


In [123]:
P1 = ["Analyze a complex computing problem and to apply principles of computing and other relevant disciplines to identify solutions."]
p1 = df1.copy()
p1.head()

,0,1
0,C1,Explain how AI impacts society
1,C2,Apply AI methods to perform practical tasks
2,C3,Synthesize a simple AI system
3,C4,Evaluate different AI approaches to solve a pr...
4,C5,Explain the basic principles of the data minin...


In [124]:
d = pd.MultiIndex.from_product(
    [q1[1], p1[1]], names=["col1", "col2"]
).to_frame(index=False)

In [125]:
d.head()

,col1,col2
0,Analyze a complex computing problem and to app...,Explain how AI impacts society
1,Analyze a complex computing problem and to app...,Apply AI methods to perform practical tasks
2,Analyze a complex computing problem and to app...,Synthesize a simple AI system
3,Analyze a complex computing problem and to app...,Evaluate different AI approaches to solve a pr...
4,Analyze a complex computing problem and to app...,Explain the basic principles of the data minin...


In [126]:
d.shape

(786, 2)

In [127]:
d.head(133)

,col1,col2
0,Analyze a complex computing problem and to app...,Explain how AI impacts society
1,Analyze a complex computing problem and to app...,Apply AI methods to perform practical tasks
2,Analyze a complex computing problem and to app...,Synthesize a simple AI system
3,Analyze a complex computing problem and to app...,Evaluate different AI approaches to solve a pr...
4,Analyze a complex computing problem and to app...,Explain the basic principles of the data minin...
...,...,...
128,Analyze a complex computing problem and to app...,Define the computing requirements appropriate ...
129,Analyze a complex computing problem and to app...,Apply knowledge of computer science theory and...
130,Analyze a complex computing problem and to app...,Develop a computer science application.
131,"Design, implement, and evaluate a computing-ba...",Explain how AI impacts society


In [128]:
from strsimpy.normalized_levenshtein import NormalizedLevenshtein 
normalized_levenshtein = NormalizedLevenshtein() 
d["leven_sim"] = [normalized_levenshtein.distance(i,j) for i,j in zip(d["col1"],d["col2"])]

In [129]:
d.head()

,col1,col2,leven_sim
0,Analyze a complex computing problem and to app...,Explain how AI impacts society,0.841270
1,Analyze a complex computing problem and to app...,Apply AI methods to perform practical tasks,0.785714
2,Analyze a complex computing problem and to app...,Synthesize a simple AI system,0.849206
3,Analyze a complex computing problem and to app...,Evaluate different AI approaches to solve a pr...,0.769841
4,Analyze a complex computing problem and to app...,Explain the basic principles of the data minin...,0.706349


In [130]:
d.tail()

,col1,col2,leven_sim
781,Apply computer science theory and software dev...,Discuss the socio-economic implications of onl...,0.771429
782,Apply computer science theory and software dev...,Function in groups to assess current ethical i...,0.857143
783,Apply computer science theory and software dev...,Define the computing requirements appropriate ...,0.657143
784,Apply computer science theory and software dev...,Apply knowledge of computer science theory and...,0.638095
785,Apply computer science theory and software dev...,Develop a computer science application.,0.761905


In [131]:
def split_dataframe(df, chunk_size = 131): 
    chunks = list()
    num_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size else 0)
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [132]:
d_split2 = split_dataframe(d, chunk_size=131)

In [133]:
d_split2

[                                                  col1  ... leven_sim
 0    Analyze a complex computing problem and to app...  ...  0.841270
 1    Analyze a complex computing problem and to app...  ...  0.785714
 2    Analyze a complex computing problem and to app...  ...  0.849206
 3    Analyze a complex computing problem and to app...  ...  0.769841
 4    Analyze a complex computing problem and to app...  ...  0.706349
 ..                                                 ...  ...       ...
 126  Analyze a complex computing problem and to app...  ...  0.738095
 127  Analyze a complex computing problem and to app...  ...  0.730159
 128  Analyze a complex computing problem and to app...  ...  0.611111
 129  Analyze a complex computing problem and to app...  ...  0.730159
 130  Analyze a complex computing problem and to app...  ...  0.785714
 
 [131 rows x 3 columns],
                                                   col1  ... leven_sim
 131  Design, implement, and evaluate a computing-

In [134]:
d_split2[0].head()

,col1,col2,leven_sim
0,Analyze a complex computing problem and to app...,Explain how AI impacts society,0.841270
1,Analyze a complex computing problem and to app...,Apply AI methods to perform practical tasks,0.785714
2,Analyze a complex computing problem and to app...,Synthesize a simple AI system,0.849206
3,Analyze a complex computing problem and to app...,Evaluate different AI approaches to solve a pr...,0.769841
4,Analyze a complex computing problem and to app...,Explain the basic principles of the data minin...,0.706349


In [135]:
d_split2[1].head()

,col1,col2,leven_sim
131,"Design, implement, and evaluate a computing-ba...",Explain how AI impacts society,0.878378
132,"Design, implement, and evaluate a computing-ba...",Apply AI methods to perform practical tasks,0.831081
133,"Design, implement, and evaluate a computing-ba...",Synthesize a simple AI system,0.878378
134,"Design, implement, and evaluate a computing-ba...",Evaluate different AI approaches to solve a pr...,0.783784
135,"Design, implement, and evaluate a computing-ba...",Explain the basic principles of the data minin...,0.783784


In [136]:
d_split2[2].head()

,col1,col2,leven_sim
262,Communicate effectively in a variety of profes...,Explain how AI impacts society,0.806452
263,Communicate effectively in a variety of profes...,Apply AI methods to perform practical tasks,0.758065
264,Communicate effectively in a variety of profes...,Synthesize a simple AI system,0.774194
265,Communicate effectively in a variety of profes...,Evaluate different AI approaches to solve a pr...,0.725806
266,Communicate effectively in a variety of profes...,Explain the basic principles of the data minin...,0.790323


In [137]:
d_split2[3].head()

,col1,col2,leven_sim
393,Recognize professional responsibilities and ma...,Explain how AI impacts society,0.843750
394,Recognize professional responsibilities and ma...,Apply AI methods to perform practical tasks,0.796875
395,Recognize professional responsibilities and ma...,Synthesize a simple AI system,0.867188
396,Recognize professional responsibilities and ma...,Evaluate different AI approaches to solve a pr...,0.765625
397,Recognize professional responsibilities and ma...,Explain the basic principles of the data minin...,0.773438


In [138]:
d_split2[4].tail()

,col1,col2,leven_sim
650,Function effectively as a member or leader of ...,Discuss the socio-economic implications of onl...,0.739130
651,Function effectively as a member or leader of ...,Function in groups to assess current ethical i...,0.730435
652,Function effectively as a member or leader of ...,Define the computing requirements appropriate ...,0.739130
653,Function effectively as a member or leader of ...,Apply knowledge of computer science theory and...,0.730435
654,Function effectively as a member or leader of ...,Develop a computer science application.,0.791304


In [139]:
d_split2[5].head()

,col1,col2,leven_sim
655,Apply computer science theory and software dev...,Explain how AI impacts society,0.819048
656,Apply computer science theory and software dev...,Apply AI methods to perform practical tasks,0.723810
657,Apply computer science theory and software dev...,Synthesize a simple AI system,0.847619
658,Apply computer science theory and software dev...,Evaluate different AI approaches to solve a pr...,0.780952
659,Apply computer science theory and software dev...,Explain the basic principles of the data minin...,0.761905


In [140]:
d_split2[0].drop(["col1", "col2"], axis=1, inplace=True)
d_split2[1].drop(["col1", "col2"], axis=1, inplace=True)
d_split2[2].drop(["col1", "col2"], axis=1, inplace=True)
d_split2[3].drop(["col1", "col2"], axis=1, inplace=True)
d_split2[4].drop(["col1", "col2"], axis=1, inplace=True)
d_split2[5].drop(["col1", "col2"], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [141]:
d_split2[0].rename(columns = {'leven_sim':'P1'}, inplace = True)
d_split2[0].head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,P1
0,0.841270
1,0.785714
2,0.849206
3,0.769841
4,0.706349


In [142]:
d_split2[1].rename(columns = {'leven_sim':'P2'}, inplace = True)
d_split2[2].rename(columns = {'leven_sim':'P3'}, inplace = True)
d_split2[3].rename(columns = {'leven_sim':'P4'}, inplace = True)
d_split2[4].rename(columns = {'leven_sim':'P5'}, inplace = True)
d_split2[5].rename(columns = {'leven_sim':'P6'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [143]:
d_split2[5].reset_index(drop=True, inplace=True)
d_split2[5].head()

,P6
0,0.819048
1,0.723810
2,0.847619
3,0.780952
4,0.761905


In [144]:
d_split2[1].reset_index(drop=True, inplace=True)
d_split2[2].reset_index(drop=True, inplace=True)
d_split2[3].reset_index(drop=True, inplace=True)
d_split2[4].reset_index(drop=True, inplace=True)

In [145]:
dd = pd.concat([d_split2[0], d_split2[1], d_split2[2], d_split2[3], d_split2[4], d_split2[5]], axis=1)
dd.head()

,P1,P2,P3,P4,P5,P6
0,0.841270,0.878378,0.806452,0.843750,0.860870,0.819048
1,0.785714,0.831081,0.758065,0.796875,0.808696,0.723810
2,0.849206,0.878378,0.774194,0.867188,0.852174,0.847619
3,0.769841,0.783784,0.725806,0.765625,0.782609,0.780952
4,0.706349,0.783784,0.790323,0.773438,0.756522,0.761905


In [146]:
dd.tail()

,P1,P2,P3,P4,P5,P6
126,0.738095,0.709459,0.764706,0.710938,0.739130,0.771429
127,0.730159,0.736486,0.759615,0.710938,0.730435,0.857143
128,0.611111,0.716216,0.738095,0.726562,0.739130,0.657143
129,0.730159,0.743243,0.709302,0.734375,0.730435,0.638095
130,0.785714,0.810811,0.790323,0.796875,0.791304,0.761905


In [147]:
df2 = pd.concat([p1[0], dd], axis=1)
df2.head()

,0,P1,P2,P3,P4,P5,P6
0,C1,0.841270,0.878378,0.806452,0.843750,0.860870,0.819048
1,C2,0.785714,0.831081,0.758065,0.796875,0.808696,0.723810
2,C3,0.849206,0.878378,0.774194,0.867188,0.852174,0.847619
3,C4,0.769841,0.783784,0.725806,0.765625,0.782609,0.780952
4,C5,0.706349,0.783784,0.790323,0.773438,0.756522,0.761905


In [148]:
df2.set_index(0, inplace=True)
df2.head()

,P1,P2,P3,P4,P5,P6
0,,,,,,
C1,0.841270,0.878378,0.806452,0.843750,0.860870,0.819048
C2,0.785714,0.831081,0.758065,0.796875,0.808696,0.723810
C3,0.849206,0.878378,0.774194,0.867188,0.852174,0.847619
C4,0.769841,0.783784,0.725806,0.765625,0.782609,0.780952
C5,0.706349,0.783784,0.790323,0.773438,0.756522,0.761905


In [149]:
df2.tail()

,P1,P2,P3,P4,P5,P6
0,,,,,,
C127,0.738095,0.709459,0.764706,0.710938,0.739130,0.771429
C128,0.730159,0.736486,0.759615,0.710938,0.730435,0.857143
C129,0.611111,0.716216,0.738095,0.726562,0.739130,0.657143
C130,0.730159,0.743243,0.709302,0.734375,0.730435,0.638095
C131,0.785714,0.810811,0.790323,0.796875,0.791304,0.761905
